In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
from support.omniwheel_calculation import *
from support.pd_support import *
from support.calculations_support import *
from support.ar_calculations import *
from support.imu_calculations import *
from numba import njit

In [2]:
_parent_folder = "omni_9dof_feb_28_2023"
_folder_name = "sk27_00_random_slow"
_base_pth = os.path.dirname(os.getcwd())
_base_pth = os.path.join(_base_pth, "..","recording_programs", "test_data", _parent_folder)
_base_pth

'c:\\Users\\CMC\\Documents\\openposelibs\\pose\\armbo\\simulation\\..\\recording_programs\\test_data\\omni_9dof_feb_28_2023'

In [3]:
_sk_df = pl.read_csv(os.path.join(_base_pth, _folder_name, "imu01.csv"))

# convert unix epoc to datetime
_sk_df = _sk_df.with_columns([pl.col("rust_time").apply(lambda x: datetime.fromtimestamp(x))])
# rename columns rust_time to time
_sk_df = _sk_df.rename({"rust_time": "time", "e_fr":"e_t"})
# _sk_df

In [7]:
df = _sk_df[['time', 'e_t', 'e_rl', "e_rr"]]

columns = ['e_t', 'e_rl', "e_rr"]

for i in columns:
    df = df.with_columns([pl.col(i).apply(lambda x: x - df[i][0])])


In [8]:
ang_per_increment = 0.09
del_t = 0.01

for name in columns:
    df = df.with_columns([pl.col(name).apply(lambda x: x * ang_per_increment).alias(name + "_angle")])
    df = df.with_columns([((pl.col(name + "_angle").diff()/ del_t)*np.pi/180).alias(name + "_av")])
    # replace null values with 0
    df = df.fill_null(0)


In [35]:
# mat = np.array([[-y, 1, 0], [-x, 0, -1], [x, 0, -1]])
radius = 1
x = 1
y = 1

mat = np.array([[-y, 1, 0], [-x, 0, -1], [x, 0, -1]])
pmat = np.linalg.pinv(mat)
my_dict = {"_vx":[],"_vy":[],"_w":[]}

for i in range(len(df)):
    val = np.array([df[columns[0]][i], df[columns[1]][i], df[columns[2]][i]]).reshape(3,1)
    # val = np.float16(val)
    res = np.dot(pmat, val) * radius
    my_dict["_w"].append(res[0][0])
    my_dict["_vx"].append(res[1][0])
    my_dict["_vy"].append(res[2][0])

# add the calculated values to the dataframe
df = df.with_columns([pl.Series(name = "vx", values = my_dict["_vx"]),
                      pl.Series(name = "vy", values = my_dict["_vy"]),
                        pl.Series(name = "w", values = my_dict["_w"])])
df

time,e_t,e_rl,e_rr,e_t_angle,e_t_av,e_rl_angle,e_rl_av,e_rr_angle,e_rr_av,vx,vy,w
datetime[μs],i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-02-28 16:41:44.878866,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.041952,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.042352,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.042665,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.042954,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.043235,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.043539,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.043851,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28 16:41:45.044149,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_pd = df.to_pandas()

mat = np.array([[-y, 1, 0], [-x, 0, -1], [x, 0, -1]])
pmat = np.linalg.pinv(mat)
_vx = []
_vy = []
_w = []

for i in range(len(df)):
    val = np.array([df_pd[columns[0]].iloc[i], df_pd[columns[1]].iloc[i], df_pd[columns[2]].iloc[i]]).reshape(3,1)
    res = np.dot(pmat, val) * radius
    _w.append(res[0][0])
    _vx.append(res[1][0])
    _vy.append(res[2][0])
df_pd["w"] = _w
df_pd["vx"] = _vx
df_pd["vy"] = _vy

In [39]:
# calculating displacement
my_dict = {"_x":[],"_y":[]}

for i in range(len(df)):
    if i == 0:
        my_dict["_x"].append(0)
        my_dict["_y"].append(0)
    else:
        my_dict["_x"].append(0.5*(df["vx"][i] + df["vx"][i-1])*0.01)
        my_dict["_y"].append(0.5*(df["vy"][i] + df["vy"][i-1])*0.01)

# add the calculated values to the dataframe
df = df.with_columns([pl.Series(name = "x", values = my_dict["_x"]),
                        pl.Series(name = "y", values = my_dict["_y"])])

In [41]:
column_name = "w"

my_dict = {"_theta":[]}
angle = 0
for i in range(len(df[column_name])):

    if i == 0:
        my_dict["_theta"].append(0)
    else:
        angle = angle + (df[column_name][i] + df[column_name][i-1])*0.01
        my_dict["_theta"].append(angle)

# add the calculated values to the dataframe
df = df.with_columns([pl.Series(name = "theta", values = my_dict["_theta"])])

#     if i == 0:
#         _angle.append(0)
#     else:
#         angle = angle + (df[column_name].iloc[i] + df[column_name].iloc[i-1])*0.01
#         _angle.append(angle)

# df["theta"] = _angle